In [862]:
import pandas as pd

In [864]:
path = 'Downloads/train.csv'

In [866]:
df = pd.read_csv(path)

In [868]:
df

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [870]:
import numpy as np
percent = 97
print(np.percentile([len(x) for x in df['dialogue']],percent))

1584.0


In [872]:
print(np.percentile([len(x) for x in df['summary']],percent))

283.0


In [874]:
max_input_seq_length = 1584
max_output_seq_length = 290

In [876]:
dialogue_list = list(df['dialogue'])

In [878]:
summary_list = list(df['summary'])

In [880]:
def is_validlength(paragraph,max_seq_length):
    return(len(paragraph)<max_input_seq_length)
valid_textsummaries_idx = []
for i in range(len(df['summary'])):
    dialogue,summary = dialogue_list[i],summary_list[i]
    if(is_validlength(dialogue,max_input_seq_length) and is_validlength(summary,max_output_seq_length)):
        valid_textsummaries_idx.append(i)

In [882]:
len(valid_textsummaries_idx)

12085

In [884]:
dialogue_list_new = []
summary_list_new = []
for i in valid_textsummaries_idx:
    dialogue_list_new.append(article_list[i])
    summary_list_new.append(highlights_list[i])

In [886]:
from torch.utils.data import Dataset,DataLoader

In [888]:
class TextDataset(Dataset):
    def __init__(self,dialogues,summaries):
        self.dialogues = dialogues
        self.summaries = summaries
    def __len__(self):
        return len(self.summaries)
    def __getitem__(self,j):
        return self.dialogues[j],self.summaries[j]

In [890]:
data = TextDataset(dialogue_list_new,summary_list_new)

In [892]:
data[0]

("#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 "Mr. Smith's getting a check-up, and Doctor Hawkins 

In [894]:
df['dialogue'][0]

"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor."

In [896]:
len(data)

12085

In [898]:
data = TextDataset(dialogue_list_new, summary_list_new)
batch_size = 2
train_dataset = DataLoader(data, batch_size=batch_size, shuffle=True)

In [900]:
c = 1
for batch in train_dataset:
    dialogues,summaries = batch
    print(summaries)
    if(c==1):
        break

('Alice tells #Person2# her Social Security number, home address, phone number, and deposits $5500 to open a temporary savings account.', '#Person1# helps #Person2# order some drinks.')


In [902]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
import torch

In [903]:
len(train_dataset)

6043

In [906]:
max = 0
for i in dialogue_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

573


In [908]:
max = 0
for i in summary_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

129


In [910]:
input_ids = []
output_ids = []
max_input_seq_length = 573
max_output_seq_length = 150
for batch in train_dataset:
    if(len(batch)==2):
        dialogues, summaries = batch 
        tokenized_dialogues = tokenizer(list(dialogues),max_length=max_input_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=False)
        tokenized_summaries = tokenizer(list(summaries),max_length=max_output_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=True)
        input_ids.append(tokenized_dialogues['input_ids'])
        output_ids.append(tokenized_summaries['input_ids'])

In [912]:
len(output_ids)

6043

In [914]:
tensor_tokenized_inputs = torch.cat(input_ids,dim=0)

In [916]:
for i in range(10):
    print(input_ids[i])

tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
      

In [918]:
tensor_tokenized_outputs = torch.cat(output_ids,dim=0)

In [920]:
tensor_tokenized_inputs

tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        ...,
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])

In [922]:
len(tensor_tokenized_inputs)

12085

In [924]:
import math
from torch import nn
import torch.nn.functional as func

In [926]:
class Multiheadattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def foward(self,x,mask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batches,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul((q,k_transpose))/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = F.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [928]:
class Layernormalisation(nn.Module):
    def __init__(self,weights__dims_shape):
        super.__init__()
        self.eps = 1e-5
        self.weights_dims_ln = weights_dims_shape
        self.gamma = nn.Parameter(torch.ones(weights_dims))
        self.beta = nn.Parameter(torch.zeros(weights_dims))
    def foward(self,inp):
        param_dim = [] 
        for i in range(len(self.weights_dims_ln)):
            param_dim.append(-i+1)
        mean = inp.mean(dim=param_dim,keepdim=True)
        var = ((inp-mean)**2).mean(dim=param_dim,keepdim=True)
        std = math.sqrt(var+eps)
        y = (inp-mean)/std
        outputs = self.gamma*y+sef.beta
        return outputs

In [930]:
class feedfowardlayer(nn.Module):
    def __init__(self,weights_dims,hidden_dim,drop_prob):
        self.weights_dims = weights_dims
        self.hidden_dims = hidden_dim
        self.linearlayer1 = nn.Linear(self.weights_dims,self.hidden_dims)
        self.relu = nn.ReLu()
        self.linearlayer = nn.Linear(self.hidden_dims,self.weights.dims)
        self.drop_probability = drop_prob
        self.drop_neurons = nn.Dropout(p=self.drop_probability)
    def foward(self,x):
        out = self.linearlayer1(x)
        out = self.relu(out)
        out = self.drop_neurons(out)
        out = self.linearlayer2(out)
        return out

In [932]:
class Subencoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subencoder,self).__init__()
        self.attention = Multiheadattention(weight_dim,n_heads)
        self.norm = Layernormalisation(params = [weight_dim])
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm2 = LayerNormalisation(params = [weight_dim])
        self.dropout2 = nn.Dropout(p = drop_probability)
    def foward(self,x,mask):
        residual = x
        x = self.attention(x,mask)
        x = self.dropout1(x)
        x = self.norm(x+residual)
        residual = x
        x = self.ffd(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual)
        return x

In [934]:
class Sequentialencoder(nn.Sequential):
    def foward(self,x,mask):
        self.x = x
        self.mask = mask
        for module in self._modules.values():
            x = module(self.x,self.mask)
        return y

In [936]:
class Encoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability,layers):
        super().__init__()
        encoded_layers = []
        for k in range(layers):
            l = Subencoder(weight_dim,ff_hidden,n_heads,drop_probability)
            encoded_layers.append(l)
        self.layers = Sequentialencoder(*[Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)])
    def foward(self,x,mask):
        x = self.layers(x,mask)
        return res

In [938]:
class Multiheadcrossattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.kv_matrices = nn.Linear(weights_dim,2*weights_dim)
        self.q_matrix = nn.Linear(weights_dim,weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def foward(self,y,x,mask):
        batches,maxseqlength,weight_dim = x.size()
        kv = self.kv_matrices(x)
        q = self.q_matrix(y)
        kv = kv.reshape(batches,maxseqlength,self.n_heads,2*self.head_dimensions)
        kv = kv.permute(0,2,1,3)
        k,v = kv.chunk(2,dim=-1)
        q = q.reshape(batches,maxseqlength,self.n_heads,self.head_dimensions)
        q = q.permute(0,2,1,3)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul((q,k_transpose))/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = F.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [942]:
class Maskedattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def foward(self,x,mask,padmask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batches,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul((q,k_transpose))/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask+padmask
        attention_scores = F.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [944]:
class Subdecoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subencoder,self).__init__()
        self.maskedattention = Maskedattention(weight_dim,n_heads)
        self.norm = Layernormalisation([weight_dim])
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.Multicrossattention = Multiheadcrossattention(weight_dim,n_heads)
        self.norm2 = Layernormalisation([weight_dim])
        self.dropout2 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm3 = Layernormalisation([weight_dims])
        self.dropout3 = nn.Dropout(p = drop_probability)
    def foward(self,x,y,mask,padmask,crossmask):
        residual = y
        y = self.maskedattention(y,mask,padmask)
        y = self.dropout1(y)
        y = self.norm(y+residual)
        residual = y
        y = self.Multicrossattention(y,x,crossmask)
        y = self.dropout2(y)
        y = self.norm2(y+residual)
        residual = y
        y = self.ffd(y)
        y = self.dropout3(y)
        y = self.norm3(y+residual)
        return y

In [960]:
class Sequentialdecoder(nn.Sequential):
    def foward(self,x,y,mask,crossattentpadmask):
        for module in self._modules.values():
            y = module(x,y,mask,crossattentpadmask)
        return y

In [962]:
class Decoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability,layers):
        super().__init__()
        decoded_layers = []
        for k in range(layers):
            l = Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)
            decoded_layers.append(l)
        self.layers = Sequentialdecoder(*[Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)])
    def foward(self,x,y,mask,crossattentpadmask):
        resdecoder = self.layers(x,y,mask,crossattentpadmask)
        return resdecoder

In [964]:
vocab_size = tokenizer.vocab_size

In [966]:
vocab_size

32000

In [968]:
class PositionalEncoding(nn.Module):
    def __init__(self, weight_dim, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.weight_dim = weight_dim
        self.positional_embeddings = nn.Parameter(torch.zeros(max_sequence_length, weight_dim), requires_grad=True)
        self.create_positional_encodings()
    def create_positional_encodings(self):
        even_i = torch.arange(0, self.weight_dim, 2).float()
        denominator = torch.pow(10000, even_i / self.weight_dim)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position/denominator)
        odd_PE = torch.cos(position/denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        self.positional_embeddings.data = torch.flatten(stacked, start_dim=1, end_dim=2)
    def forward(self):
        return self.positional_embeddings

In [970]:
def to_one_hot(token_ids, num_classes):
    one_hot = torch.zeros(token_ids.size(0), token_ids.size(1), num_classes, dtype=torch.float32)
    one_hot.scatter_(2, token_ids.unsqueeze(2), 1) 
    return one_hot

In [ ]:
class Transformermodel(nn.Module):
    def __init__(self, encoder, decoder):
        super(TransformerModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self,src,tgt,encoder_padmask,decoder_lookaheadmask,decoder_crossattentpadmask):
        x = self.encoder(src,encoder_padmask)
        decoder_output = self.decoder(x,tgt,decoder_lookaheadmask,decoder_crossattentpadmask)
        return decoder_output
transformer_model = TransformerModel(encoder, decoder)

In [666]:
for i in range(30):
    weights_dim = 512
    ff_hidden = 2048
    n_heads = 8
    drop_probability = 0.1
    num_layers = 3
    max_seq_length_input = 573
    max_seq_length_output = 150
    batch_size = 2
    embedding_matrix_inputs = nn.Linear(31933,512)
    embedding_matrix_outputs = nn.Linear(31933,512)
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        current_batch_inputs = tensor_tokenized_inputs[start_idx:end_idx]  
        current_batch_outputs = tensor_tokenized_outputs[start_idx:end_idx]
    with torch.no_grad(): 
        X = to_one_hot(current_batch_inputs,31933) 
    with torch.no_grad():
        Y = to_one_hot(current_batch_outputs,31933)
    embedded_X = linear_layer(X)
    embedded_Y = linear_layer(Y)
    positional_embeddings = get_positional_embeddings(573, 512)
    positional_embeddings = positional_embeddings.unsqueeze(0).expand(X.size(0), -1, -1) 
    final_X = embedded_X + positional_embeddings
    positional_embeddings = get_positional_embeddings(573, 512)
    positional_embeddings = positional_embeddings.unsqueeze(0).expand(Y.size(0), -1, -1) 
    final_Y = embedded_Y + positional_embeddings
    

SyntaxError: invalid syntax (2230674481.py, line 11)

In [633]:
tensor_tokenized_inputs

tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        ...,
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])

In [974]:
start_idx = 8 * 2
end_idx = start_idx + 2
current_batch = tensor_tokenized_inputs[start_idx:end_idx]  
with torch.no_grad(): 
    x = to_one_hot(current_batch,31933) 
print((i + 1, x.shape))

(10, torch.Size([2, 573, 31933]))


In [740]:
print(torch.max(tensor_tokenized_inputs))

tensor(31993)
